In [1]:
import cx_Oracle

ModuleNotFoundError: No module named 'cx_Oracle'

In [ ]:
import os
os.putenv('NLS_LANG','.UTF8')

connection = cx_Oracle.connect('hr','1234','localhost/xe')
cursor = connection.cursor()

In [ ]:
import requests

# API 인증 정보
client_id = "htGbu9KOTEU69iArqr2A"
client_secret = "kUr1gumsmm"

# header에 추가될 내용
headers = {'X-Naver-Client-Id':client_id, 'X-Naver-Client-Secret':client_secret}

In [ ]:
url1 = "https://openapi.naver.com/v1/search/blog?query=강남역"

In [ ]:
# url 주소에 header를 포함한 GET 요청을 보냄
result = requests.get(url1, headers=headers)

In [ ]:
# 검색결과 information 변수에 넣기
information = result.json()

In [ ]:
total = information["total"]
print("총 검색 결과: ", total)

In [ ]:
total_page = 0
if total % 20 == 0:
    total_page = total // 20
else:
    total_page = total // 20+1

In [ ]:
print("전체 페이지: ", total_page)

In [ ]:
def get_api_result(keyword, display, start):
    url = "https://openapi.naver.com/v1/search/blog?query=" + keyword + "&display=" + str(display) + "&start=" + str(start)
    print("url", url)
    result = requests.get(url,headers=headers)
    return result.json()

In [ ]:
get_api_result("강남역",20,21)

In [ ]:
get_api_result("강남역",20,41)

In [ ]:
def call_and_print(keyword, page):
    json_object = get_api_result(keyword, 20, (page-1)*20+1)
    for item in json_object["items"]:
        # <title 다음 '<b>' 지워줌>
        title = item['title'].replace("<b>","").replace("'", "&quot;")
        description = item['description'].replace("<b>","").replace("</b>","").replace("'","&quot;")
        bloggername = item['bloggername'].replace("'", "&quot;").replace("</b>","")
        link = item['link']
        print('title:',title, ':description:',description,":bloggername:",bloggername,":link:",link)
        # insert_sql 쿼리 생성
        insert_sql = "insert into blog_info values(blog_sql.nextval,'{}','{}','{}','{}')".format(title, description, bloggername, link)
        print("insert_sql", insert_sql)
        # insert 쿼리 실행
        cursor.execute(insert_sql)
        # 커밋
        connection.commit()

In [ ]:
import time

In [ ]:
keyword = "강남역"

for page in range(1, 1000):
    print(page, 'page')
    call_and_print(keyword, page)
    time.sleep(0.5)

In [ ]:
import cx_Oracle

import os
os.putenv('NLS_LANG', '.UTF8')

connection = cx_Oracle.connect('hr','1234','localhost/xe')
cursor = connection.cursor()

In [ ]:
import pandas as pd

In [ ]:
blog_df = pd.read_sql("select * from blog_info",connection)

In [ ]:
blog_df.head()

In [ ]:
import matplotlib.pyplot as plt

% matplotlib inline

from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname = "c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family = font_name)

In [2]:
import re
# ^ 패턴에 맞는 문자만 리턴
# \w : 숫자, 영어, 한글
# \s : 공백
pattern = "[^\w\s]"
# re.sub(pattern, repl: 패턴이 아닐 시 바꿀 값, 원본)
blog_text = [re.sub(pattern = pattern, repl= "", string = text).replace("강남","").replace("역","") for text in blog_df["DESCRIPTION"]]
blog_text

NameError: name 'blog_df' is not defined

In [ ]:
!pip install konlpy

In [ ]:
# 단어와 단어끼리 분리
import nltk
from konlpy.tag import Twitter
twitter = Twitter()

tag_list = []
for text in blog_text:
    tagged = twitter.pos(text, stem = True)
    print("tagged:", tagged)
    for tag in tagged:
        print("tag:",tag[0])
        print("tag:",tag[1])
        if(tag[1] == "Noun")|(tag[1] == "Adjective"):
            tag_list.append(tag[0])
            
    print("="*100)